# Exploratory Data Analysis on F1 using SQL
Queries covered in this notebook:
1. Finding locations that hosted maximum number of race
2. Finding circuits that hosted opening races
3. Number of races hosted by every country
4. All-time constructor points and leaders
5. Number of races organized per season
6. Comparison of all-time top 10 teams (based on points and races won)
7. Number of races won by top 10 drivers
8. Finding the fastest laps on each circuit
9. Number of constructors from each country
10. Finding the fastest qualification times on every circuit for pole position
11. Drivers who have won at least one race

Importing Libraries

In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import os 

Establishing Connection Between The SQLITE3 Database 

In [2]:
conn = sqlite3.connect("F1.db")

Reading CSV Files

In [6]:
data_1 = pd.read_csv("Dataset\circuits.csv")
data_2 = pd.read_csv("Dataset\constructor_results.csv")
data_3 = pd.read_csv("Dataset\constructor_standings.csv")
data_4 = pd.read_csv("Dataset\constructors.csv")
data_5 = pd.read_csv("Dataset\driver_standings.csv")
data_6 = pd.read_csv("Dataset\drivers.csv")
data_7 = pd.read_csv("Dataset\lap_times.csv")
data_8 = pd.read_csv("Dataset\pit_stops.csv")
data_9 = pd.read_csv("Dataset\qualifying.csv")
data_10 = pd.read_csv("D:\\Programming\\Projects\\SQL & Tableau\\F1 EDA using SQL & Tableau\\Dataset\\races.csv")
data_11 = pd.read_csv("Dataset\seasons.csv")
data_12 = pd.read_csv("Dataset\status.csv")
data_13 = pd.read_csv("D:\\Programming\\Projects\\SQL & Tableau\\F1 EDA using SQL & Tableau\\Dataset\\results.csv")
data_14 = pd.read_csv("Dataset\sprint_results.csv")

Inserting Data Into SQL Tables 

In [7]:
data_1.to_sql("circuits", conn)
data_2.to_sql("constructor_results", conn)
data_3.to_sql("constructor_standings", conn)
data_4.to_sql("constructors", conn)
data_5.to_sql("driver_standings", conn)
data_6.to_sql("drivers", conn)
data_7.to_sql("lap_times", conn)
data_8.to_sql("pit_stops", conn)
data_9.to_sql("qualifying", conn)
data_10.to_sql("races", conn)
data_11.to_sql("seasons", conn)
data_12.to_sql("status", conn)
data_13.to_sql("results", conn)
data_14.to_sql("sprint_results", conn)

180